Car License Plate Reader - Overview
1. Input Image
2. Grayscale Image
3. Binarize Image
4. Plate Localization
5. Character Segmentation
6. Character Recognition

In [126]:
# Author: Luke O'Shea Scanlan, Panagiotis Bampilis
# Module: Image Processing 2023-2024
# Group Project

import re
import cv2
import easyocr
import numpy as np
from matplotlib import pyplot as plt

In [137]:
"""
This project aims to give the user the ability to input any sample images of car license plates
and return the results as a text output.

The product is capable of recognising car license plates rotated, as well as a certain amount of blurriness.
"""

# Input Image
I = cv2.imread("Sample Inputs/personal_three.jpg")
# I = cv2.imread("Sample Inputs/personal_two.jpg")
# I = cv2.imread("Sample Inputs/personal.jpg")
# I = cv2.imread("Sample Inputs/ambulance.jpg")
# I = cv2.imread("Sample Inputs/gardai_cars.jpg")

# HS(V) Image
V = cv2.cvtColor(I, cv2.COLOR_BGR2HSV)[:,:,2]

# Binarize Image
B = cv2.adaptiveThreshold(V, maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 7, C = 0)

# Plate Localization
blank = np.zeros_like(I)
cropping_coord = []

contours, hierarchy = cv2.findContours(image=B, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
image_copy = cv2.cvtColor(I.copy(), cv2.COLOR_BGR2RGB)

for contour in contours:
 
    epsilon = 0.04 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    if len(approx) == 4: 
        ct_area = cv2.contourArea(contour)
        
        width_one = int(np.linalg.norm(approx[0][0] - approx[1][0]))
        height_one = int(np.linalg.norm(approx[1][0] - approx[2][0]))
        
        width_two = int(np.linalg.norm(approx[2][0] - approx[3][0]))
        height_two = int(np.linalg.norm(approx[3][0] - approx[0][0]))
        
        if height_one > width_one:
            height_one, width_one, height_two, width_two = width_one, height_one, width_two, height_two
        
        ck_similar_heights = abs(height_one - height_two) < 45
        ck_height_width_ratio = (height_one * 3) < width_one
        ck_similar_area_lower = ct_area > (height_one*width_one/1.1)
        ck_similar_area_upper = ct_area < (height_one*width_one*1.1) 
        ck_too_small = ct_area > 500

        total_checks = ck_too_small and ck_similar_area_lower and ck_similar_area_upper and ck_similar_heights and ck_height_width_ratio
        
        if total_checks:
            cv2.drawContours(blank, [approx], 0, (255, 255, 255), -1)
            cropping_coord.append((cv2.boundingRect(contour)))
                
                
# Plate Cropping
if len(cropping_coord) > 0:
    reg = cv2.bitwise_and(I, I, mask=cv2.cvtColor(blank, cv2.COLOR_BGR2GRAY))
    x, y, w, h = cropping_coord[0]
    CROP = reg[y:y+h,x:x+w]


# Instance of reader to detect roman characters
reader = easyocr.Reader(['en'], gpu=True)

# Roman character Recognition after morphology applied or Inverted Threshold
textBox = reader.readtext(CROP)
# print(textBox)

# Boolean to be used for pattern recognition
found = False

# Drawing box around text areas
for t in textBox:
    # print(t)
    bbox, text, score = t

    # cv2.rectangle(image_copy, bbox[0], bbox[2], (0, 255, 0), 5)
    
    # Pattern that checks through for valid recognized license plates
    if found == False:
        if re.match('\w\w-', text):
            print("License plate found on this image is: ", text)
            found = True
            break
        elif re.match('\w\w ', text):
            print("License plate found on this image is: ", text)
            found = True
            break
        elif re.match('\w\w\s-', text):
            print("License plate found on this image is: ", text)
            found = True
            break
        elif re.match('\w\w\w-', text):
            print("License plate found on this image is: ", text)
            found = True
            break
        elif re.match('\w\w\w\s-', text):
            print("License plate found on this image is: ", text)
            found = True
            break

# If License Plates in image not found
if found == False:
    print("Licenses Plates not found!")

# Show Cropped Licence Plate
fig2, arg = plt.subplots(1, 1)
arg.imshow(CROP)